In [47]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
import time
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor

# Importando Data

In [163]:

url_data= 'https://raw.githubusercontent.com/Daniel11182092/LAB2_StatisticalL2/main/supermarket_sales.csv'
dataset = pd.read_csv(url_data)

# Analisis de la Data

In [164]:
dataset.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [165]:
print('Validando cantidad de datos')
print(dataset.shape)

Validando cantidad de datos
(1000, 17)


In [166]:
print('Validando Tipos de datos')
print(dataset.info())

Validando Tipos de datos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Invoice ID               1000 non-null   object 
 1   Branch                   1000 non-null   object 
 2   City                     1000 non-null   object 
 3   Customer type            1000 non-null   object 
 4   Gender                   1000 non-null   object 
 5   Product line             1000 non-null   object 
 6   Unit price               1000 non-null   float64
 7   Quantity                 1000 non-null   int64  
 8   Tax 5%                   1000 non-null   float64
 9   Total                    1000 non-null   float64
 10  Date                     1000 non-null   object 
 11  Time                     1000 non-null   object 
 12  Payment                  1000 non-null   object 
 13  cogs                     1000 non-null   float64
 14  

In [167]:
print('Validando datos faltantes')
print(pd.isnull(dataset).sum())

Validando datos faltantes
Invoice ID                 0
Branch                     0
City                       0
Customer type              0
Gender                     0
Product line               0
Unit price                 0
Quantity                   0
Tax 5%                     0
Total                      0
Date                       0
Time                       0
Payment                    0
cogs                       0
gross margin percentage    0
gross income               0
Rating                     0
dtype: int64


In [168]:
print('Validando estadisticas de los dataset')
print(dataset.describe())

Validando estadisticas de los dataset
        Unit price     Quantity       Tax 5%        Total        cogs  \
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.00000   
mean     55.672130     5.510000    15.379369   322.966749   307.58738   
std      26.494628     2.923431    11.708825   245.885335   234.17651   
min      10.080000     1.000000     0.508500    10.678500    10.17000   
25%      32.875000     3.000000     5.924875   124.422375   118.49750   
50%      55.230000     5.000000    12.088000   253.848000   241.76000   
75%      77.935000     8.000000    22.445250   471.350250   448.90500   
max      99.960000    10.000000    49.650000  1042.650000   993.00000   

       gross margin percentage  gross income      Rating  
count              1000.000000   1000.000000  1000.00000  
mean                  4.761905     15.379369     6.97270  
std                   0.000000     11.708825     1.71858  
min                   4.761905      0.508500     4.00000  
25%      

## Preparando los datos

In [169]:
dataset['Branch'].value_counts()

A    340
B    332
C    328
Name: Branch, dtype: int64

In [170]:
dataset['City'].value_counts()

Yangon       340
Mandalay     332
Naypyitaw    328
Name: City, dtype: int64

In [171]:
dataset['Customer type'].value_counts()

Member    501
Normal    499
Name: Customer type, dtype: int64

In [172]:
dataset['Product line'].value_counts()

Fashion accessories       178
Food and beverages        174
Electronic accessories    170
Sports and travel         166
Home and lifestyle        160
Health and beauty         152
Name: Product line, dtype: int64

In [173]:
dataset['Payment'].value_counts()

Ewallet        345
Cash           344
Credit card    311
Name: Payment, dtype: int64

In [174]:
dataset['Gender'].replace(['Female','Male'],[0,1], inplace= True)
dataset['Branch'].replace(['A','B','C'],[0,1,2], inplace= True)
dataset['City'].replace(['Yangon','Mandalay','Naypyitaw'],[0,1,2], inplace= True)
dataset['Customer type'].replace(['Member','Normal'],[0,1], inplace= True)
dataset['Payment'].replace(['Ewallet','Cash','Credit card'],[0,1,2], inplace= True)
dataset['Product line'].replace(['Fashion accessories','Food and beverages','Electronic accessories','Sports and travel','Home and lifestyle','Health and beauty'],[0,1,2,3,4,5], inplace= True)

In [175]:
dataset.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,0,0,0,0,5,74.69,7,26.1415,548.9715,1/5/2019,13:08,0,522.83,4.761905,26.1415,9.1
1,226-31-3081,2,2,1,0,2,15.28,5,3.8200,80.2200,3/8/2019,10:29,1,76.40,4.761905,3.8200,9.6
2,631-41-3108,0,0,1,1,4,46.33,7,16.2155,340.5255,3/3/2019,13:23,2,324.31,4.761905,16.2155,7.4
3,123-19-1176,0,0,0,1,5,58.22,8,23.2880,489.0480,1/27/2019,20:33,0,465.76,4.761905,23.2880,8.4
4,373-73-7910,0,0,1,1,3,86.31,7,30.2085,634.3785,2/8/2019,10:37,0,604.17,4.761905,30.2085,5.3


In [176]:
dataset.drop(['Date'],axis=1 , inplace=True)
dataset.drop(['Time'],axis=1 , inplace=True)
dataset.drop(['Tax 5%'],axis=1 , inplace=True)
dataset.drop(['gross margin percentage'],axis=1 , inplace=True)


In [177]:
datasetn = dataset.iloc[:, dataset.columns != 'Invoice ID']
ID = dataset.loc[:, 'Invoice ID']

In [178]:
datasetn.head()

,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Total,Payment,cogs,gross income,Rating
0,0,0,0,0,5,74.69,7,548.9715,0,522.83,26.1415,9.1
1,2,2,1,0,2,15.28,5,80.2200,1,76.40,3.8200,9.6
2,0,0,1,1,4,46.33,7,340.5255,2,324.31,16.2155,7.4
3,0,0,0,1,5,58.22,8,489.0480,0,465.76,23.2880,8.4
4,0,0,1,1,3,86.31,7,634.3785,0,604.17,30.2085,5.3


# Dividiendo la Data

In [179]:
train, test = train_test_split(datasetn, test_size = 0.20)
test_original = test
train_original = train 
print("Datos para entrenar: ", len(train_original))
print("Datos test: ", len(test_original))

Datos para entrenar:  800
Datos test:  200


In [190]:
X = train_original.iloc[:, train_original.columns != 'Quantity']
y = train_original.loc[:, 'Quantity']
X_Test2 = test_original.loc[:, train_original.columns != 'Quantity']

In [181]:
std_scaler = StandardScaler()
X = std_scaler.fit_transform(X)

In [182]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=1234)

In [183]:
X_train.shape[1]

11

In [184]:
len(y_train.value_counts())

10

**ANN**

In [185]:
std_scaler = StandardScaler()

X_train = std_scaler.fit_transform(X_train)
X_test = std_scaler.transform(X_test)

X_train.shape, X_test.shape

((560, 11), (240, 11))

In [186]:
entradas = X_train.shape[1]
lr = 0.05
batch_sz=100

regressor = Sequential()
regressor.add(Dense(input_dim=entradas, units=21, kernel_initializer='uniform', activation='relu'))
regressor.add(Dense(units=14, kernel_initializer='uniform', activation='relu'))
regressor.add(Dense(units=7, kernel_initializer='uniform', activation='relu'))
regressor.add(Dense(units=1, kernel_initializer='uniform', activation='relu'))
regressor.compile(optimizer=Adam(lr), loss='mean_squared_error', metrics=['mean_squared_error']) #regresion.
result = regressor.fit(X_train, y_train, batch_size=batch_sz, validation_split=0.2, verbose=1, epochs=1000)

Epoch 1/1000
5/5 [==============================] - 1s 46ms/step - loss: 35.4128 - mean_squared_error: 35.4128 - val_loss: 15.2857 - val_mean_squared_error: 15.2857
Epoch 2/1000
5/5 [==============================] - 0s 8ms/step - loss: 10.8373 - mean_squared_error: 10.8373 - val_loss: 8.1170 - val_mean_squared_error: 8.1170
Epoch 3/1000
5/5 [==============================] - 0s 8ms/step - loss: 8.5745 - mean_squared_error: 8.5745 - val_loss: 7.6449 - val_mean_squared_error: 7.6449
Epoch 4/1000
5/5 [==============================] - 0s 8ms/step - loss: 6.4270 - mean_squared_error: 6.4270 - val_loss: 3.5671 - val_mean_squared_error: 3.5671
Epoch 5/1000
5/5 [==============================] - 0s 8ms/step - loss: 3.2340 - mean_squared_error: 3.2340 - val_loss: 2.9639 - val_mean_squared_error: 2.9639
Epoch 6/1000
5/5 [==============================] - 0s 8ms/step - loss: 3.0168 - mean_squared_error: 3.0168 - val_loss: 2.4578 - val_mean_squared_error: 2.4578
Epoch 7/1000
5/5 [===============

In [ ]:
y_preds = result.predict(X_test)
y_preds

In [ ]:
y_preds2 = result.predict(X_Test2)
y_preds2